In [2]:
import pandas as pd
import numpy as np
import requests

Retrieve Location data for Houston Neighborhoods with Zipcodes

In [3]:
data = 'https://web.har.com/zipcode'

Clean table from webpage and only pull in Zip Code, City, and County

In [4]:
r = requests.get(data)
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
df = df_list[2]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df = df.drop(df.columns[3], axis=1)
df = df.astype({'Zip Code':'int64'})
df = df.rename(columns ={"City  (City Alias Name(s))":"City"})
df

,Zip Code,City,County
1,77002,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",HARRIS
2,77003,"Houston - Inner Loop (HOUSTON,)",HARRIS
3,77004,"Houston - Inner Loop (HOUSTON,)",HARRIS
4,77005,"Houston - Inner Loop (HOUSTON,SOUTHSIDE PLACE,...",HARRIS
5,77006,"Houston - Inner Loop (HOUSTON,)",HARRIS
...,...,...,...
215,77591,TEXAS CITY,GALVESTON
216,77598,"WEBSTER (FONDREN,WEBSTER,)",HARRIS
217,77617,"GILCHRIST (CAPLEN,GILCHRIST,)",GALVESTON
218,77623,HIGH ISLAND,GALVESTON


Downloaded a csv file of the lat long associated with zipcodes into my github repository

In [5]:
geospat_data = 'https://raw.githubusercontent.com/jcrandall01/Coursera_Capstone/main/us-zip-code-latitude-and-longitude.csv'
geo_data = pd.read_csv(geospat_data, sep =';')
geo_data = geo_data.drop(['City','State','Timezone','Daylight savings time flag','geopoint'],axis=1)
#geo_data.head()
geo_data


,Zip,Latitude,Longitude
0,48834,43.141649,-85.04948
1,55304,45.254715,-93.28652
2,55422,45.014764,-93.33965
3,29079,34.296064,-80.11319
4,29390,34.888237,-81.96902
...,...,...,...
43186,50157,41.738800,-92.56155
43187,65212,38.937608,-92.33043
43188,11935,41.012868,-72.47230
43189,63539,39.893011,-92.74503


Joining my datasets

In [93]:
df = df.join(geo_data.set_index('Zip'), on='Zip Code')
df = df.dropna()
df

,Zip Code,City,County,Latitude,Longitude
1,77002,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",HARRIS,29.755578,-95.36531
2,77003,"Houston - Inner Loop (HOUSTON,)",HARRIS,29.749278,-95.34741
3,77004,"Houston - Inner Loop (HOUSTON,)",HARRIS,29.728779,-95.36570
4,77005,"Houston - Inner Loop (HOUSTON,SOUTHSIDE PLACE,...",HARRIS,29.717529,-95.42821
5,77006,"Houston - Inner Loop (HOUSTON,)",HARRIS,29.741878,-95.38944
...,...,...,...,...,...
215,77591,TEXAS CITY,GALVESTON,29.389583,-94.99167
216,77598,"WEBSTER (FONDREN,WEBSTER,)",HARRIS,29.539581,-95.13467
217,77617,"GILCHRIST (CAPLEN,GILCHRIST,)",GALVESTON,29.507243,-94.52028
218,77623,HIGH ISLAND,GALVESTON,29.571106,-94.40450


Foursquare API

In [54]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\jcrandal\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.1               |   py38haa244fe_0         3.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  conda                               4.10.0-py38haa244fe_1 --> 4.10.1-py38haa244fe_0




conda-4.10.1         | 3.1 MB    |            |   0% 
conda-4.10.1         | 3.1 MB    |            |   1% 
conda-4.10.1         | 3.1 MB    | #          |  10% 
conda-4.10.1         | 3.1 MB    | ####4      |  45% 
conda-4.10.1         | 3.1 MB    | ########7  |  88% 
conda-4.10.1         | 3.1 MB    | ########## | 100% 
Preparing transaction: ...working... do

In [55]:
CLIENT_ID = 'YQOIZIYEKYWWHSEPX2KBPNFPZOXZNXRQ1NMMUCAIHQP3RSF0' # your Foursquare ID
CLIENT_SECRET = 'MIRMP4WMGDW5X4X4SEKUVTWGZBOBLJD21HFBAVFIRMNRSXTU' # your Foursquare Secret
ACCESS_TOKEN = 'XS3QT3WAVLKDOY3EDSVDZM2COAHPAWX0PU35FXPDVHNWMCQV' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YQOIZIYEKYWWHSEPX2KBPNFPZOXZNXRQ1NMMUCAIHQP3RSF0
CLIENT_SECRET:MIRMP4WMGDW5X4X4SEKUVTWGZBOBLJD21HFBAVFIRMNRSXTU


In [56]:
address = 'Houston, Texas'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [94]:
# create map of Houston using latitude and longitude values
map_Houston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['City']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Houston)  

map_Houston

Now we get all venues in Houston

In [102]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [103]:
Houston_venues = getNearbyVenues(names=df['City'],
                                  latitudes=df['Latitude'],
                                  longitudes=df['Longitude'])

Houston - Inner Loop (HOUSTON,CLUTCH CITY,)
Houston - Inner Loop (HOUSTON,)
Houston - Inner Loop (HOUSTON,)
Houston - Inner Loop (HOUSTON,SOUTHSIDE PLACE,WEST UNIVERSITY PLACE,W UNIV PL,)
Houston - Inner Loop (HOUSTON,)
Houston - Inner Loop (HOUSTON,)
Houston - Inner Loop (HEIGHTS,HOUSTON,HOUSTON HEIGHTS,)
Houston - Inner Loop (HOUSTON,)
Houston - Inner Loop (HOUSTON,)
Houston - Inner Loop (HOUSTON,)
Houston - Inner Loop (HOUSTON,)
Houston - Northeast (HOUSTON,)
Houston - Northwest (HOUSTON,)
Houston - Northeast (CLOVERLEAF,GREENS BAYOU,HOUSTON,)
Houston - Northeast (HOUSTON,)
Houston - Southeast (HOUSTON,)
Houston - Northwest (HOUSTON,)
Houston - Inner Loop (HOUSTON,)
Houston - Inner Loop (HOUSTON,)
Houston - Inner Loop (HOUSTON,)
Houston - Northeast (HOUSTON,)
Houston - Inner Loop (HOUSTON,)
Houston - Southwest (BUNKER HILL VILLAGE,HEDWIG VILLAGE,HOUSTON,HUNTERS CREEK VILLAGE,MEMORIAL PARK,PINEY POINT,SPRING VALLEY,)
Houston - Inner Loop (ASTRODOME,ASTROWORLD,HOUSTON,)
Houston - Inne

In [104]:
Houston_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",29.755578,-95.36531,Pappas Bros. Steakhouse,29.755545,-95.363126,Steakhouse
1,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",29.755578,-95.36531,House of Blues,29.753822,-95.363953,Music Venue
2,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",29.755578,-95.36531,Jason's Deli,29.757464,-95.365543,Food Truck
3,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",29.755578,-95.36531,Bombay Pizza Co.,29.757700,-95.364586,Pizza Place
4,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",29.755578,-95.36531,Kelsey-Seybold Clinic - Downtown at The Shops,29.755546,-95.362629,Doctor's Office


We group venues by city

In [105]:
Houston_venues.groupby('City').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
ALVIN,10,10,10,10,10,10
BACLIFF,5,5,5,5,5,5
BAYTOWN,14,14,14,14,14,14
BELLAIRE,2,2,2,2,2,2
BRAZORIA,2,2,2,2,2,2
...,...,...,...,...,...,...
"TOMBALL (ROSE HILL,TOMBALL,)",4,4,4,4,4,4
WALLER,2,2,2,2,2,2
"WEBSTER (FONDREN,WEBSTER,)",30,30,30,30,30,30


Analyze each City

In [205]:
# one hot encoding
Houston_onehot = pd.get_dummies(Houston_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
Houston_onehot['City'] = Houston_venues['City'] 

# move city column to the first column
fixed_columns = [Houston_onehot.columns[-1]] + list(Houston_onehot.columns[:-1])
Houston_onehot = Houston_onehot[fixed_columns]

pd.set_option('display.max_columns', None)
Houston_onehot

,City,ATM,Accessories Store,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beer Garden,Big Box Store,Bistro,Board Shop,Boat Launch,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Caribbean Restaurant,Casino,Chinese Restaurant,Clothing Store,Coffee Shop,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Event Service,Event Space,Eye Doctor,Fabric Shop,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Medical School,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightlife Spot,Non-Profit,Noodle House,Office,Optical Shop,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Pool,Post Office,Print Shop,Professional & Other Places,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Ski Area,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Swim School,Szechuan Restaurant,Taco Place,Tanning Salon,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Trail,Travel & Transport,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wings Joint,Women's Store,Yoga Studio
0,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Grouping Categories and taking the mean frequency of each

In [216]:
Houston_grouped = Houston_onehot.groupby('City').mean().reset_index()
#Houston_onehot = Houston_onehot.sort_values(by='Brewery',ascending=False)
#Houston_onehot

Only need to consider "Brewery", "Bar", or "Beer Garden" venue types

In [217]:
Houston_brew = Houston_grouped[['City','Brewery','Bar','Beer Garden']]
Houston_brew

,City,Brewery,Bar,Beer Garden
0,ALVIN,0.0,0.0,0.0
1,BACLIFF,0.0,0.0,0.0
2,BAYTOWN,0.0,0.0,0.0
3,BELLAIRE,0.0,0.0,0.0
4,BRAZORIA,0.0,0.0,0.0
...,...,...,...,...
85,"TOMBALL (ROSE HILL,TOMBALL,)",0.0,0.0,0.0
86,WALLER,0.0,0.0,0.0
87,"WEBSTER (FONDREN,WEBSTER,)",0.0,0.0,0.0
88,"WEST COLUMBIA (EAST COLUMBIA,WEST COLUMBIA,)",0.0,0.0,0.0


In [218]:
num_top_venues = 5

for hood in Houston_brew['City']:
    print("----"+hood+"----")
    temp = Houston_brew[Houston_brew['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALVIN----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----BACLIFF----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----BAYTOWN----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----BELLAIRE----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----BRAZORIA----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----CHANNELVIEW----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----CLUTE----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----CONROE----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----CONROE (BEACH,CAMP STRAKE,CONROE,LAKELAND,TAMINA,)----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----CROSBY----
         venue  freq
0 

2  Beer Garden   0.0


----SANTA FE (ARCADIA,SANTA FE,)----
         venue  freq
0          Bar   1.0
1      Brewery   0.0
2  Beer Garden   0.0


----SEABROOK (EL LAGO,SEABROOK,TAYLOR LAKE VILLAGE,TIMBER COVE,TOD,TAYLOR LK VLG,)----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----SOUTH HOUSTON----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----SPLENDORA (PATTON,SPLENDORA,)----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----SPRING----
         venue  freq
0          Bar  0.03
1      Brewery  0.00
2  Beer Garden  0.00


----SPRING (KLEIN,SPRING,)----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----SPRING (RAYFORD,SPRING,)----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----STAFFORD----
         venue  freq
0      Brewery   0.0
1          Bar   0.0
2  Beer Garden   0.0


----SUGAR

Taking the output and putting into a Pandas dataframe

In [219]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [220]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] = Houston_brew['City']

for ind in np.arange(Houston_brew.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Houston_brew.iloc[ind, :], num_top_venues)

city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,ALVIN,Beer Garden,Bar,Brewery
1,BACLIFF,Beer Garden,Bar,Brewery
2,BAYTOWN,Beer Garden,Bar,Brewery
3,BELLAIRE,Beer Garden,Bar,Brewery
4,BRAZORIA,Beer Garden,Bar,Brewery


Now we begin clustering the neighborhoods using K-means

In [221]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

Houston_brew_clustering = Houston_brew.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Houston_brew_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:3] 

array([0, 0, 0])

In [187]:
#df
city_venues_sorted

,Cluster Labels,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,0,ALVIN,Beer Garden,Bar,Brewery
1,0,BACLIFF,Beer Garden,Bar,Brewery
2,0,BAYTOWN,Beer Garden,Bar,Brewery
3,0,BELLAIRE,Beer Garden,Bar,Brewery
4,0,BRAZORIA,Beer Garden,Bar,Brewery
...,...,...,...,...,...
85,0,"TOMBALL (ROSE HILL,TOMBALL,)",Beer Garden,Bar,Brewery
86,0,WALLER,Beer Garden,Bar,Brewery
87,0,"WEBSTER (FONDREN,WEBSTER,)",Beer Garden,Bar,Brewery
88,0,"WEST COLUMBIA (EAST COLUMBIA,WEST COLUMBIA,)",Beer Garden,Bar,Brewery


In [222]:
# add clustering labels
city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Houston_merged = df

# merge Houston_brew with df to add latitude/longitude for each city
Houston_merged = Houston_merged.join(city_venues_sorted.set_index('City'), on='City')

Houston_merged.head() # check the last columns!

,Zip Code,City,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,77002,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",HARRIS,29.755578,-95.36531,3.0,Bar,Beer Garden,Brewery
2,77003,"Houston - Inner Loop (HOUSTON,)",HARRIS,29.749278,-95.34741,3.0,Bar,Beer Garden,Brewery
3,77004,"Houston - Inner Loop (HOUSTON,)",HARRIS,29.728779,-95.36570,3.0,Bar,Beer Garden,Brewery
4,77005,"Houston - Inner Loop (HOUSTON,SOUTHSIDE PLACE,...",HARRIS,29.717529,-95.42821,0.0,Beer Garden,Bar,Brewery
5,77006,"Houston - Inner Loop (HOUSTON,)",HARRIS,29.741878,-95.38944,3.0,Bar,Beer Garden,Brewery


Mapping Cluster Results

In [223]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Houston_merged = Houston_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Houston_merged['Latitude'], Houston_merged['Longitude'], Houston_merged['City'], Houston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

Cluster 1

In [224]:
Houston_merged.loc[Houston_merged['Cluster Labels'] == 0, Houston_merged.columns[[1] + list(range(5, Houston_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
4,"Houston - Inner Loop (HOUSTON,SOUTHSIDE PLACE,...",0.0,Beer Garden,Bar,Brewery
12,"Houston - Northeast (HOUSTON,)",0.0,Beer Garden,Bar,Brewery
13,"Houston - Northwest (HOUSTON,)",0.0,Bar,Beer Garden,Brewery
15,"Houston - Northeast (HOUSTON,)",0.0,Beer Garden,Bar,Brewery
16,"Houston - Southeast (HOUSTON,)",0.0,Brewery,Beer Garden,Bar
...,...,...,...,...,...
213,SOUTH HOUSTON,0.0,Beer Garden,Bar,Brewery
214,"TEXAS CITY (MESKILL,TEXAS CITY,WEST TEXAS CITY,)",0.0,Beer Garden,Bar,Brewery
215,TEXAS CITY,0.0,Beer Garden,Bar,Brewery
216,"WEBSTER (FONDREN,WEBSTER,)",0.0,Beer Garden,Bar,Brewery


Cluster 2

In [225]:
Houston_merged.loc[Houston_merged['Cluster Labels'] == 1, Houston_merged.columns[[1] + list(range(5, Houston_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
176,"SANTA FE (ARCADIA,SANTA FE,)",1.0,Bar,Beer Garden,Brewery


Cluster 3

In [226]:
Houston_merged.loc[Houston_merged['Cluster Labels'] == 2, Houston_merged.columns[[1] + list(range(5, Houston_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
107,"DOBBIN (BOBVILLE,DOBBIN,)",2.0,Beer Garden,Bar,Brewery


Cluster 4

In [227]:
Houston_merged.loc[Houston_merged['Cluster Labels'] == 3, Houston_merged.columns[[1] + list(range(5, Houston_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,"Houston - Inner Loop (HOUSTON,CLUTCH CITY,)",3.0,Bar,Beer Garden,Brewery
2,"Houston - Inner Loop (HOUSTON,)",3.0,Bar,Beer Garden,Brewery
3,"Houston - Inner Loop (HOUSTON,)",3.0,Bar,Beer Garden,Brewery
5,"Houston - Inner Loop (HOUSTON,)",3.0,Bar,Beer Garden,Brewery
6,"Houston - Inner Loop (HOUSTON,)",3.0,Bar,Beer Garden,Brewery
8,"Houston - Inner Loop (HOUSTON,)",3.0,Bar,Beer Garden,Brewery
9,"Houston - Inner Loop (HOUSTON,)",3.0,Bar,Beer Garden,Brewery
10,"Houston - Inner Loop (HOUSTON,)",3.0,Bar,Beer Garden,Brewery
11,"Houston - Inner Loop (HOUSTON,)",3.0,Bar,Beer Garden,Brewery
18,"Houston - Inner Loop (HOUSTON,)",3.0,Bar,Beer Garden,Brewery


Cluster 5

In [228]:
Houston_merged.loc[Houston_merged['Cluster Labels'] == 4, Houston_merged.columns[[1] + list(range(5, Houston_merged.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
7,"Houston - Inner Loop (HEIGHTS,HOUSTON,HOUSTON ...",4.0,Beer Garden,Bar,Brewery
204,LEAGUE CITY,4.0,Beer Garden,Bar,Brewery


Cluster 4 (teal) Has the highest denisty of establishments while Cluster 1 (red) has the lowest

Ideally, I would not place a brewery in a dense brewery area, nor around bars and beer gardens